In [ ]:
# !pip install xlrd
# !pip install sklearn
# !pip install pandas

In [28]:
import pandas as pd

#use uglyunemployment if you don't want the preprocessing/organization of the table to happen in excel, if you open the fil you will see i made a new sheet with a cleaned table which essentially does what we do here 
uglyunemployment = pd.read_excel('unemployment_rate.xls', sheet_name=1, skiprows=7, header=None)

# Use the 6th row as column headers - assigns values in the first row to the columns 
uglyunemployment.columns = uglyunemployment.iloc[0]

# Drop the duplicated header row (which is now the first row)
uglyunemployment = uglyunemployment.iloc[4]
uglyunemployment = pd.DataFrame(uglyunemployment).dropna(how="all")
uglyunemployment = uglyunemployment.iloc[1:] #select all rows after the second one, the first one was a header
uglyunemployment.index.name = "index"
uglyunemployment.columns = ["PT: Portugal"]

uglyunemployment

,PT: Portugal
index,
3.º Trimestre de 2023,6.1
2.º Trimestre de 2023,6.1
1.º Trimestre de 2023,7.2
4.º Trimestre de 2022,6.6
3.º Trimestre de 2022,6
2.º Trimestre de 2022,5.9
1.º Trimestre de 2022,6.1
4.º Trimestre de 2021,6.4
3.º Trimestre de 2021,6.2


In [ ]:
unemployment = pd.read_excel('unemployment_rate.xls') 
# Extracting the year and quarter information from "Unnamed: 0"
# Extract the first value from "Unnamed: 0" and convert to integer
#then use a small formula to go from trimester value to month value M = (T-1)*3 + 1
unemployment['MONTH'] = (unemployment["Unnamed: 0"].str.extract('(\\d+)').astype(int)-1)*3 + 1

# Extract the last 4 characters from "Unnamed: 0" and convert to integer
unemployment['YEAR'] = unemployment["Unnamed: 0"].str[-4:].astype(int)


# Combine "YEAR" and "MONTH" columns to create a new datetime column
unemployment['datetime_column'] = pd.to_datetime(unemployment[['YEAR', 'MONTH']].assign(day=1))
unemployment.set_index('datetime_column', inplace = True)
unemployment.drop(columns=['Unnamed: 0'], inplace = True)

#For the resample of the dates later we need to add a copy of the most recent value and add 12 weeks to it
# resample ffill works such that it copies up to the last value and we want to continue for the entire trimester  
# Create a new DataFrame with the values to be added
new_row = pd.DataFrame([[6.1, 7, 2023]], columns=unemployment.columns)

# Determine the last date in the dataset
last_date = unemployment.index.max()

new_row.index = [pd.to_datetime(last_date)]
# Add 12 weeks to the date value
new_row.index = new_row.index + pd.DateOffset(weeks=12)
#Add the new row
unemployment = pd.concat([unemployment, new_row])
# Determine the last date in the dataset
last_date = unemployment.index.max()

#make the data weekly, add a week column and rename the data column
unemployment = unemployment.resample("W").ffill() #resamples the time-series data at a weekly frequency
unemployment['WEEK'] = unemployment.index.isocalendar().week #extracts the week
unemployment['DATA'] = unemployment['PT: Portugal']
unemployment.drop(columns=['PT: Portugal'], inplace=True)

# Format "TIME_KEY" based on "YEAR" and "WEEK" with leading zero and Insert "TIME_KEY" as the first column
unemployment.insert(0, 'TIME_KEY', unemployment.apply(lambda row: f"{int(row['YEAR'])}{int(row['WEEK']):02}", axis=1))
#we dont need the date values in the index anymore
unemployment.reset_index(drop = True, inplace=True)

unemployment


In [42]:
from datetime import datetime
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Prepare the features (X) and target variable (y)
X = unemployment[['MONTH', 'YEAR', 'WEEK']]
y = unemployment['DATA']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions for the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

# Get the current date and time
today = datetime.now().date()

# Generate future dates weekly for 4 months (16 weeks) after the last date
#Only need to add 2 months since from setemeber to november we have the data
future_dates = pd.date_range(start=last_date, end=today + pd.DateOffset(months=2), freq='W')

# Create a DataFrame for future prediction
future_data = pd.DataFrame({
    'datetime_column': future_dates,
    'MONTH': future_dates.month,
    'YEAR': future_dates.year,
    'WEEK': future_dates.isocalendar().week,
})

# Make predictions for the future data
future_data['DATA'] = model.predict(future_data[['MONTH', 'YEAR', 'WEEK']])

future_data.drop(columns=['datetime_column'], inplace=True)

# Format "TIME_KEY" based on "YEAR" and "WEEK" with leading zero and Insert "TIME_KEY" as the first column
future_data.insert(0, 'TIME_KEY', future_data.apply(lambda row: f"{int(row['YEAR'])}{int(row['WEEK']):02}", axis=1))
#we dont need the date values in the index anymore
future_data.reset_index(drop = True, inplace=True)

future_data


Mean Squared Error: 0.1885067359048713


,TIME_KEY,MONTH,YEAR,WEEK,DATA
0,202338,9,2023,38,6.004551
1,202339,10,2023,39,5.937924
2,202340,10,2023,40,5.938886
3,202341,10,2023,41,5.939847
4,202342,10,2023,42,5.940809
5,202343,10,2023,43,5.941770
6,202344,11,2023,44,5.875144
7,202345,11,2023,45,5.876105
8,202346,11,2023,46,5.877066
9,202347,11,2023,47,5.878028


In [43]:
#Add the predicted data
unemployment = pd.concat([unemployment, future_data])
unemployment

,TIME_KEY,MONTH,YEAR,WEEK,DATA
0,202153,1,2021,53,7.300000
1,202101,1,2021,1,7.300000
2,202102,1,2021,2,7.300000
3,202103,1,2021,3,7.300000
4,202104,1,2021,4,7.300000
...,...,...,...,...,...
14,202352,12,2023,52,5.815247
15,202401,1,2024,1,6.340129
16,202402,1,2024,2,6.341091
17,202403,1,2024,3,6.342052
